In [7]:
!pip install tensorflow
!pip install keras
!pip install kaggle

In [5]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [9]:
! mkdir ~/.kaggle

In [10]:
!cp /content/drive/MyDrive/CollabData/kaggle_API/kaggle.json ~/.kaggle/kaggle.json

In [11]:
! chmod 600 ~/.kaggle/kaggle.json

In [12]:
! kaggle datasets download samuelcortinhas/sports-balls-multiclass-image-classification

Dataset URL: https://www.kaggle.com/datasets/samuelcortinhas/sports-balls-multiclass-image-classification
License(s): CC0-1.0
 94% 379M/401M [00:04<00:00, 109MB/s]
100% 401M/401M [00:04<00:00, 98.9MB/s]


In [13]:
! unzip sports-balls-multiclass-image-classification.zip

Output streaming akan dipotong hingga 5000 baris terakhir.
  inflating: train/cricket_ball/cricket_ball_182.jpg  
  inflating: train/cricket_ball/cricket_ball_183.jpg  
  inflating: train/cricket_ball/cricket_ball_185.jpg  
  inflating: train/cricket_ball/cricket_ball_186.jpg  
  inflating: train/cricket_ball/cricket_ball_187.jpg  
  inflating: train/cricket_ball/cricket_ball_189.jpg  
  inflating: train/cricket_ball/cricket_ball_19.jpg  
  inflating: train/cricket_ball/cricket_ball_190.jpg  
  inflating: train/cricket_ball/cricket_ball_191.jpg  
  inflating: train/cricket_ball/cricket_ball_192.jpg  
  inflating: train/cricket_ball/cricket_ball_195.jpg  
  inflating: train/cricket_ball/cricket_ball_196.jpg  
  inflating: train/cricket_ball/cricket_ball_199.jpg  
  inflating: train/cricket_ball/cricket_ball_200.jpg  
  inflating: train/cricket_ball/cricket_ball_201.jpg  
  inflating: train/cricket_ball/cricket_ball_202.jpg  
  inflating: train/cricket_ball/cricket_ball_209.jpg  
  infla

In [19]:
# Create an ImageDataGenerator for preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess the training and testing datasets
train_generator = train_datagen.flow_from_directory(
    '/content/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    '/content/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 7328 images belonging to 15 classes.
Found 1841 images belonging to 15 classes.


In [22]:
# Load the pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Determine the number of classes (sports ball types) in your dataset
num_classes = 15

# Add a new classification layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(num_classes, activation='softmax')(x) # num_classes is now defined

# Create the final model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)

Epoch 1/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4703 - loss: 1.7768

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


229/229 ━━━━━━━━━━━━━━━━━━━━ 471s 2s/step - accuracy: 0.4710 - loss: 1.7745 - val_accuracy: 0.7670 - val_loss: 0.7742
Epoch 2/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 503s 2s/step - accuracy: 0.7752 - loss: 0.7131 - val_accuracy: 0.7862 - val_loss: 0.6750
Epoch 3/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 522s 2s/step - accuracy: 0.8207 - loss: 0.5871 - val_accuracy: 0.7944 - val_loss: 0.6455
Epoch 4/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 464s 2s/step - accuracy: 0.8548 - loss: 0.4747 - val_accuracy: 0.8065 - val_loss: 0.6184
Epoch 5/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 501s 2s/step - accuracy: 0.8710 - loss: 0.4357 - val_accuracy: 0.8158 - val_loss: 0.5841
Epoch 6/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 466s 2s/step - accuracy: 0.8749 - loss: 0.4126 - val_accuracy: 0.8120 - val_loss: 0.5887
Epoch 7/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 562s 2s/step - accuracy: 0.8982 - loss: 0.3469 - val_accuracy: 0.8218 - val_loss: 0.5879
Epoch 8/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 564s 2s/step - accuracy: 0.9053 - loss: 0.3237 - val_accuracy: 0.830